In [1]:
import time
import string
import json
import pandas as pd
import nltk
import spacy
from nltk.corpus import stopwords
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop
from nltk.stem.snowball import SnowballStemmer

nlp = spacy.load("fr_core_news_sm")
stemmer = SnowballStemmer(language='french')

def ouvrir_json(chemin):
    f = open(chemin, encoding="utf-8")
    toto = json.load(f)
    f.close()
    return toto

In [2]:
ligneRATP_twitter = ouvrir_json("LigneRATP_twitter.json")
causes = ouvrir_json("causes.json")
consequences = ouvrir_json("consequences.json")

In [3]:
start = time.time()
df = pd.read_csv('/Users/mustaphalazreg/Desktop/RATP/2018_2021_twitter_database_serenive.csv')
end = time.time()
print("-"*60)
print(f"le temps d'exécution: {end - start} secondes")

/Users/mustaphalazreg/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (18,21,36,37,38,41,44,45,46) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


------------------------------------------------------------
le temps d'exécution: 813.1251578330994 secondes


In [4]:
#le noms des columns 
print('Les noms des colonnes:', df.columns)
print("-"*25)
#print the shape 
print('Le nombre de ligne & colonne:',df.shape)

Les noms des colonnes: Index(['Unnamed: 0', 'id', 'source', 'time', 'tweets', 'retweet_count',
       'reply_count', 'like_count', 'quote_count', 'tweet_link', 'language',
       'conversation_id', 'referenced_type', 'in_reply_to_tweet_id',
       'in_reply_to_user_id', 'in_reply_to_user_name', 'in_quoted_tweet_id',
       'in_quoted_user_id', 'in_quoted_user_name', 'media_type', 'media_url',
       'expanded_url', 'mention_usernames', 'mention_ids', 'mention_count',
       'author_name', 'author_id', 'author_created', 'author_verified',
       'author_description', 'author_location', 'author_url',
       'author_followers_count', 'author_following_count',
       'author_tweet_count', 'author_listed_count', 'geo_place_id', 'geo_name',
       'geo_bbox', 'retrieved_at', 'tweets_0', 'tweets_0_repeat', 'tweets_1',
       'end_search_point', 'type', 'type_real', 'entry', 'Unnamed: 0.1'],
      dtype='object')
-------------------------
Le nombre de ligne & colonne: (13901168, 48)


## Récupération des en-têtes et sous-en-têtes  pour chaque arbre conversationnel.


In [5]:
df_head = df[df['conversation_id'] == df['id']]
print(df_head.shape)
df_list_head = []
for ligne in ligneRATP_twitter:
    df_list_head.append(df_head[df_head.author_name == ligne])
df_head_v1 = pd.concat(df_list_head,ignore_index=False)
df_head_v1 = df_head_v1.drop_duplicates(["id"])
print(df_head_v1.shape)

(769581, 48)
(65565, 48)


In [ ]:
(769581, 48)
(65885, 48)

In [6]:
df_reply = df[df['in_reply_to_user_name'] == df['author_name']]
print(df_reply.shape)
df_list_replied = []
for ligne in ligneRATP_twitter:
    df_list_replied.append(df_reply[df_reply.in_reply_to_user_name == ligne])
df_replied = pd.concat(df_list_replied,ignore_index=False)
df_replied = df_replied.drop_duplicates(["id"])
print(df_replied.shape)

(757614, 48)
(21264, 48)


In [7]:
df_final = pd.concat([df_replied,df_head_v1])

In [8]:
df_final.shape

(86829, 48)

In [11]:
df_final.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis = 1)

,id,source,time,tweets,retweet_count,reply_count,like_count,quote_count,tweet_link,language,...,geo_name,geo_bbox,retrieved_at,tweets_0,tweets_0_repeat,tweets_1,end_search_point,type,type_real,entry
447118,1464733963713499145,Hootsuite Inc.,2021-11-28 00:12:45,[✅ Incident d'exploitation] 00:18- Le trafic e...,0,0,2,0,https://twitter.com/user/status/14647339637134...,fr,...,NaN,NaN,2021-11-30 16:00:20,NaN,NaN,[✅ Incident d'exploitation] 00:18- Le trafic e...,2021-11-28 00:00:00,from:RER_A,RERA,NaN
447120,1464710344144785410,Hootsuite Inc.,2021-11-27 22:38:53,[✅ difficultés d'exploitation] 22:38- Le trafi...,2,0,5,0,https://twitter.com/user/status/14647103441447...,fr,...,NaN,NaN,2021-11-30 16:00:26,NaN,NaN,[✅ difficultés d'exploitation] 22:38- Le trafi...,2021-11-28 00:00:00,from:RER_A,RERA,NaN
447122,1464697653879574531,Hootsuite Inc.,2021-11-27 21:48:28,[✅ difficultés d'exploitation] 21:48 - Le traf...,0,0,2,0,https://twitter.com/user/status/14646976538795...,fr,...,NaN,NaN,2021-11-30 16:00:32,NaN,NaN,[✅ difficultés d'exploitation] 21:48 - Le traf...,2021-11-28 00:00:00,from:RER_A,RERA,NaN
447125,1464307008975384582,Hootsuite Inc.,2021-11-26 19:56:11,Fin de stationnement à Houilles–Carrières-sur-...,0,0,2,0,https://twitter.com/user/status/14643070089753...,fr,...,NaN,NaN,2021-11-30 16:00:44,NaN,NaN,Fin de stationnement à Houilles–Carrières-sur-...,2021-11-28 00:00:00,from:RER_A,RERA,NaN
447127,1464304935747600384,Hootsuite Inc.,2021-11-26 19:47:56,Fin de stationnement à Achères–Ville. Le train...,0,0,3,0,https://twitter.com/user/status/14643049357476...,fr,...,NaN,NaN,2021-11-30 16:00:50,NaN,NaN,Fin de stationnement à Achères–Ville. Le train...,2021-11-28 00:00:00,from:RER_A,RERA,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13646337,956074052388642816,Hootsuite,2018-01-24 08:59:51,08:55 Reprise progressive du trafic sur l'ense...,1,1,0,0,https://twitter.com/user/status/95607405238864...,fr,...,NaN,NaN,2022-01-25 17:55:23,NaN,NaN,08:55 Reprise progressive du trafic sur l'ense...,2018-09-01 00:00:00,@Ligne10_RATP,Ligne10RATP,NaN
13646341,956022802074750977,Hootsuite,2018-01-24 05:36:12,"Toute la journée, en raison de #CrueSeine, le ...",2,2,1,0,https://twitter.com/user/status/95602280207475...,fr,...,NaN,NaN,2022-01-25 17:55:30,NaN,NaN,"Toute la journée, en raison de #CrueSeine, le ...",2018-09-01 00:00:00,@Ligne10_RATP,Ligne10RATP,NaN
13646372,955812221745008643,Hootsuite,2018-01-23 15:39:26,La station Saint Michel Notre Dame RER C est d...,2,1,0,0,https://twitter.com/user/status/95581222174500...,fr,...,NaN,NaN,2022-01-25 17:55:51,NaN,NaN,La station Saint Michel Notre Dame RER C est d...,2018-09-01 00:00:00,@Ligne10_RATP,Ligne10RATP,NaN
13880063,956566315828989952,Hootsuite,2018-01-25 17:35:56,"En raison de #CrueSeine, le trafic @RERC_SNCF ...",1,1,0,0,https://twitter.com/user/status/95656631582898...,fr,...,NaN,NaN,2022-01-29 13:37:30,NaN,NaN,"En raison de #CrueSeine, le trafic @RERC_SNCF ...",2018-09-01 00:00:00,@ClientsRATP,ClientsRATP,NaN


In [12]:
df_final.to_csv("df_final.csv")

In [10]:
#df_final.shape

(87525, 48)

In [3]:
start = time.time()
df_final = pd.read_csv('/Users/mustaphalazreg/Desktop/RATP/Untitled Folder/df_final.csv')
end = time.time()
print("-"*60)
print(f"le temps d'exécution: {end - start} secondes")

------------------------------------------------------------
le temps d'exécution: 1.6292390823364258 secondes


/Users/mustaphalazreg/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (16,37,38,39) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### Méthode 1 pour récupérer les tweets + causes/conséquences

In [265]:
df_causes = []
start = time.time()
for tweet in df_final['tweets'].to_numpy(): 
    for element in causes:
        output = set(return_stem(element)).issubset(return_stem(tweet))
        if output == True:
            df_ = pd.DataFrame({'tweets': [tweet],'causes': [element]})
            df_causes.append(df_)
            
end = time.time()
print("-"*60)
print(f"le temps d'exécution: {end - start} secondes")

------------------------------------------------------------
le temps d'exécution: 89676.47211003304 secondes


In [257]:
df_consequences = []
start = time.time()
for tweet in df_final['tweets'].to_numpy(): 
    for element in consequences:
        output = set(return_stem(element)).issubset(return_stem(tweet))
        if output == True:
            df_ = pd.DataFrame({'tweets': [tweet],'consequences': [element]})
            df_consequences.append(df_)
            
end = time.time()
print("-"*60)
print(f"le temps d'exécution: {end - start} secondes")

------------------------------------------------------------
le temps d'exécution: 55499.005887031555 secondes


In [266]:
df__causes = pd.concat(df_causes1)

In [269]:
df__causes.shape

(56798, 2)

In [267]:
df_final_caus = pd.merge(df_final, df__causes, on = ['tweets'])

In [268]:
df_final_caus.to_csv("df_final_caus.csv")

In [259]:
df__consequences = pd.concat(df_consequences)

In [279]:
df_final_cons = pd.merge(df_final, df__consequences, on = ['tweets'])

In [261]:
df_final_cons.to_csv("df_final_cons.csv")

In [281]:
df_final_caus = pd.read_csv('df_final_caus.csv')
df_final_cons = pd.read_csv('df_final_cons.csv')

/Users/mustaphalazreg/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (13,16,19,37,38,39) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/Users/mustaphalazreg/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (13,16,19,24,37,38,39) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### Méthode 2 pour récupérer les tweets + causes/conséquences

In [29]:
def chercherCaus0(x):
    for y in causes:
        if set(return_stem(y)).issubset(return_stem(x)):
            return y
        
def chercherCons0(x):
    for y in consequences:
        if set(return_stem(y)).issubset(return_stem(x)):
            return y

In [17]:
start = time.time()

df_final["causes"] = list(map(chercherCaus0, df_final['tweets']))

end = time.time()
print("-"*60)
print(f"le temps d'exécution: {end - start} secondes")

------------------------------------------------------------
le temps d'exécution: 70257.47360992432 secondes


In [30]:
start = time.time()

df_final["consequences"] = list(map(chercherCons0, df_final['tweets']))

end = time.time()
print("-"*60)
print(f"le temps d'exécution: {end - start} secondes")

------------------------------------------------------------
le temps d'exécution: 48883.33360695839 secondes


In [34]:
df_final = df_final.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis = 1)

In [36]:
df_final.to_csv("df_headers_caus_cons")

### Méthode 3 nettoyage avant la racinisation

In [4]:
mot_vide = stopwords.words('french')+list(fr_stop)
mot_vide = list(set(mot_vide))
#new_mv = ["c’","s’","j’","d’","l’","m’","n’","qu’","t’","quelqu’un"]
mot_vide_supp = ['autres','autre','diverses','diverse','divers']
mots_vides = []
for i in mot_vide:
    if i not in mot_vide_supp:
        mots_vides.append(i)
len(mots_vides)

574

In [5]:
def nettoyage(texte):
    sans_mv = [word.text.lower() for word in nlp(texte) if word.text.lower() not in mots_vides] #1 supprimer les stopwords
    mot_propre = [element for element in sans_mv if element not in string.punctuation] #2 supprimer les ponctuations
    return mot_propre #3 retouner la liste de mots propres

def racinisation(texte):
    return [stemmer.stem(x) for x in nettoyage(texte)]


In [6]:
def chercherCaus(x):
    for y in causes:
        if set(racinisation(y)).issubset(racinisation(x)):
            return y
        
def chercherCons(x):
    for y in consequences:
        if set(racinisation(y)).issubset(racinisation(x)):
            return y

In [9]:
start = time.time()

df_final["causes"] = list(map(chercherCaus, df_final['tweets_1']))

end = time.time()
print("-"*60)
print(f"le temps d'exécution: {end - start} secondes")

------------------------------------------------------------
le temps d'exécution: 85541.26662397385 secondes


In [13]:
start = time.time()

df_final["consequences"] = list(map(chercherCons, df_final['tweets']))

end = time.time()
print("-"*60)
print(f"le temps d'exécution: {end - start} secondes")

------------------------------------------------------------
le temps d'exécution: 48851.933212041855 secondes


In [29]:
#df_final = df_final.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'], axis = 1)
df_final["time"] = pd.to_datetime(df_final["time"])
df_final.to_csv('df_headers_caus_cons_v2.csv')

In [28]:
df_final

,id,source,time,tweets,retweet_count,reply_count,like_count,quote_count,tweet_link,language,...,retrieved_at,tweets_0,tweets_0_repeat,tweets_1,end_search_point,type,type_real,entry,causes,consequences
0,1464733963713499145,Hootsuite Inc.,2021-11-28 00:12:45,[✅ Incident d'exploitation] 00:18- Le trafic e...,0,0,2,0,https://twitter.com/user/status/14647339637134...,fr,...,2021-11-30 16:00:20,NaN,NaN,[✅ Incident d'exploitation] 00:18- Le trafic e...,2021-11-28 00:00:00,from:RER_A,RERA,NaN,Incident d'exploitation,Trafic rétabli
1,1464710344144785410,Hootsuite Inc.,2021-11-27 22:38:53,[✅ difficultés d'exploitation] 22:38- Le trafi...,2,0,5,0,https://twitter.com/user/status/14647103441447...,fr,...,2021-11-30 16:00:26,NaN,NaN,[✅ difficultés d'exploitation] 22:38- Le trafi...,2021-11-28 00:00:00,from:RER_A,RERA,NaN,Difficultés d'exploitation,Trafic rétabli
2,1464697653879574531,Hootsuite Inc.,2021-11-27 21:48:28,[✅ difficultés d'exploitation] 21:48 - Le traf...,0,0,2,0,https://twitter.com/user/status/14646976538795...,fr,...,2021-11-30 16:00:32,NaN,NaN,[✅ difficultés d'exploitation] 21:48 - Le traf...,2021-11-28 00:00:00,from:RER_A,RERA,NaN,Difficultés d'exploitation,Trafic rétabli
3,1464307008975384582,Hootsuite Inc.,2021-11-26 19:56:11,Fin de stationnement à Houilles–Carrières-sur-...,0,0,2,0,https://twitter.com/user/status/14643070089753...,fr,...,2021-11-30 16:00:44,NaN,NaN,Fin de stationnement à Houilles–Carrières-sur-...,2021-11-28 00:00:00,from:RER_A,RERA,NaN,Acte de malveillance,Train repart
4,1464304935747600384,Hootsuite Inc.,2021-11-26 19:47:56,Fin de stationnement à Achères–Ville. Le train...,0,0,3,0,https://twitter.com/user/status/14643049357476...,fr,...,2021-11-30 16:00:50,NaN,NaN,Fin de stationnement à Achères–Ville. Le train...,2021-11-28 00:00:00,from:RER_A,RERA,NaN,Incident d'exploitation,Train repart
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86824,956074052388642816,Hootsuite,2018-01-24 08:59:51,08:55 Reprise progressive du trafic sur l'ense...,1,1,0,0,https://twitter.com/user/status/95607405238864...,fr,...,2022-01-25 17:55:23,NaN,NaN,08:55 Reprise progressive du trafic sur l'ense...,2018-09-01 00:00:00,@Ligne10_RATP,Ligne10RATP,NaN,None,Reprise progressive du trafic
86825,956022802074750977,Hootsuite,2018-01-24 05:36:12,"Toute la journée, en raison de #CrueSeine, le ...",2,2,1,0,https://twitter.com/user/status/95602280207475...,fr,...,2022-01-25 17:55:30,NaN,NaN,"Toute la journée, en raison de #CrueSeine, le ...",2018-09-01 00:00:00,@Ligne10_RATP,Ligne10RATP,NaN,None,Trafic interrompu
86826,955812221745008643,Hootsuite,2018-01-23 15:39:26,La station Saint Michel Notre Dame RER C est d...,2,1,0,0,https://twitter.com/user/status/95581222174500...,fr,...,2022-01-25 17:55:51,NaN,NaN,La station Saint Michel Notre Dame RER C est d...,2018-09-01 00:00:00,@Ligne10_RATP,Ligne10RATP,NaN,None,None
86827,956566315828989952,Hootsuite,2018-01-25 17:35:56,"En raison de #CrueSeine, le trafic @RERC_SNCF ...",1,1,0,0,https://twitter.com/user/status/95656631582898...,fr,...,2022-01-29 13:37:30,NaN,NaN,"En raison de #CrueSeine, le trafic @RERC_SNCF ...",2018-09-01 00:00:00,@ClientsRATP,ClientsRATP,NaN,None,Trafic interrompu
